## adding FC layer and stacking RNN

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
sentence = "if you wnat to build a ship, don't drum up peaple together to "
sentence += "collect wood and don't assign them tasks and work, but rather "
sentence += "teach them to long for the endless immensity of the sea."

# make dictionary
char_set = list(set(sentence))
char_dict = {c: i for i, c in enumerate(char_set)}
# hyper parameters
input_size = len(char_dict)
hidden_size = len(char_dict)
learning_rate = 0.1
seq_length = 10

# data setting
x_data = []
y_data = []
for i in range(len(sentence)-seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1:i + seq_length + 1]
    print(i, x_str, '->', y_str)
    x_data.append([char_dict[c] for c in x_str])
    y_data.append([char_dict[c] for c in y_str])
    
x_one_hot = [np.eye(input_size)[x] for x in x_data]

0 if you wna -> f you wnat
1 f you wnat ->  you wnat 
2  you wnat  -> you wnat t
3 you wnat t -> ou wnat to
4 ou wnat to -> u wnat to 
5 u wnat to  ->  wnat to b
6  wnat to b -> wnat to bu
7 wnat to bu -> nat to bui
8 nat to bui -> at to buil
9 at to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up pea
36

In [5]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [11]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super().__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)
        
    def forward(self, x):
        x, _satus = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(input_size, hidden_size, 2)

In [12]:
# loss & optimizer setting
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(axis=2)
    predict_str = ""
    for j, result in enumerate(results):
        print(i,j, ''.join([char_set[c] for c in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[c] for c in result])
        else:
            predict_str += char_set[result[-1]]
#     result_str = ''.join([char_set[c] for c in np.squeeze(result)])
#     print(f"[Epoch: {i}]", "loss: {:.4f}".format(loss.item()), "\nprediction:", result, "\ntreu Y:", y_data, "\nprediction str: ", result_str)

0 0 ccllullglo 3.1959359645843506
0 1 nccullglol 3.1959359645843506
0 2 ncullgloll 3.1959359645843506
0 3 cullglollc 3.1959359645843506
0 4 nclglollcn 3.1959359645843506
0 5 clllollcnl 3.1959359645843506
0 6 nclollcnlc 3.1959359645843506
0 7 'lcllcnlcl 3.1959359645843506
0 8 lcllcnlcll 3.1959359645843506
0 9 iclcnlclll 3.1959359645843506
0 10 iccnlcllll 3.1959359645843506
0 11 ncllclllll 3.1959359645843506
0 12 iclcllllll 3.1959359645843506
0 13 ncllllllll 3.1959359645843506
0 14 ncllllllll 3.1959359645843506
0 15 lulllllllc 3.1959359645843506
0 16 ccllllllcc 3.1959359645843506
0 17 cclllllccu 3.1959359645843506
0 18 clllllccul 3.1959359645843506
0 19 nclllccull 3.1959359645843506
0 20 ncllcculln 3.1959359645843506
0 21 illccullnl 3.1959359645843506
0 22 nccculllll 3.1959359645843506
0 23 tccullllll 3.1959359645843506
0 24 nculllllll 3.1959359645843506
0 25 cullnlllln 3.1959359645843506
0 26 nclnllllnc 3.1959359645843506
0 27 lccllllnlc 3.1959359645843506
0 28 ncllllnlcl 3.195935964584

3 0 ee,mumum'p 3.4019336700439453
3 1 se,myp''.' 3.4019336700439453
3 2 se,m'pdw,. 3.4019336700439453
3 3  e,m'rse,m 3.4019336700439453
3 4  e,murde,m 3.4019336700439453
3 5  m.'nrt.yp 3.4019336700439453
3 6 smypdm...' 3.4019336700439453
3 7 se,m.'nm.. 3.4019336700439453
3 8 sm,m'pdw.' 3.4019336700439453
3 9 ee,mursmym 3.4019336700439453
3 10 se,mum.'ym 3.4019336700439453
3 11 sm,m'mum,. 3.4019336700439453
3 12 se,m'pde,m 3.4019336700439453
3 13  e,mntee,m 3.4019336700439453
3 14 smymnesmym 3.4019336700439453
3 15 sm,m e mym 3.4019336700439453
3 16   ,m mdmnm 3.4019336700439453
3 17 ee murswym 3.4019336700439453
3 18 eeemumnm,m 3.4019336700439453
3 19 se,m'pnt,. 3.4019336700439453
3 20 se,m'mde,m 3.4019336700439453
3 21 ee,mneem,r 3.4019336700439453
3 22 sm.mnm.''r 3.4019336700439453
3 23 se,mnmupd. 3.4019336700439453
3 24 eeemurdeem 3.4019336700439453
3 25 ee,murdehm 3.4019336700439453
3 26 ee,mdrem,p 3.4019336700439453
3 27 se,mdmd..p 3.4019336700439453
3 28 sm,m'm'''' 3.401933670043

7 0  dt dttd t 3.0027236938476562
7 1 d o  tt  o 3.0027236938476562
7 2 dot dot d  3.0027236938476562
7 3 noo d  dtt 3.0027236938476562
7 4 ndt do dot 3.0027236938476562
7 5   tt d t d 3.0027236938476562
7 6 dot d t d  3.0027236938476562
7 7 adt do dtt 3.0027236938476562
7 8 do  o  dt  3.0027236938476562
7 9 not d t  t 3.0027236938476562
7 10 ott do  tt 3.0027236938476562
7 11 dot d  dot 3.0027236938476562
7 12 odt do  tt 3.0027236938476562
7 13 ndt dot do 3.0027236938476562
7 14 do  do  tt 3.0027236938476562
7 15     o  tt  3.0027236938476562
7 16   tt dtt d 3.0027236938476562
7 17  do  tt do 3.0027236938476562
7 18 dtt d   tt 3.0027236938476562
7 19 odt do dot 3.0027236938476562
7 20 dot do  tt 3.0027236938476562
7 21 ndt do  tt 3.0027236938476562
7 22 dot do  tt 3.0027236938476562
7 23 d   tt d t 3.0027236938476562
7 24     tt dot 3.0027236938476562
7 25  do  tt dt 3.0027236938476562
7 26      tt to 3.0027236938476562
7 27 ntt do dot 3.0027236938476562
7 28 dot do  tt 3.002723693847

10 35 o s     t  2.85241961479187
10 36        t   2.85241961479187
10 37    t  t  t 2.85241961479187
10 38      t  t  2.85241961479187
10 39 e   t      2.85241961479187
10 40    t  t  t 2.85241961479187
10 41         t  2.85241961479187
10 42 e          2.85241961479187
10 43            2.85241961479187
10 44   t  t   h 2.85241961479187
10 45   t      h 2.85241961479187
10 46            2.85241961479187
10 47   h        2.85241961479187
10 48   t  t     2.85241961479187
10 49 e          2.85241961479187
10 50      t   h 2.85241961479187
10 51 n h        2.85241961479187
10 52            2.85241961479187
10 53   t  t   h 2.85241961479187
10 54            2.85241961479187
10 55            2.85241961479187
10 56   t   h  t 2.85241961479187
10 57          h 2.85241961479187
10 58 e       t  2.85241961479187
10 59   t   h    2.85241961479187
10 60 n h      h 2.85241961479187
10 61 e        h 2.85241961479187
10 62         t  2.85241961479187
10 63 n t        2.85241961479187
10 64   t   h 

13 128  e  t      2.7977592945098877
13 129 t    t   t 2.7977592945098877
13 130     et  t  2.7977592945098877
13 131  e     t   2.7977592945098877
13 132   t  tt  t 2.7977592945098877
13 133   t  t  t  2.7977592945098877
13 134 t   t  tt  2.7977592945098877
13 135   t  t  t  2.7977592945098877
13 136 nrt  t   e 2.7977592945098877
13 137 t t  t   t 2.7977592945098877
13 138 t e     tt 2.7977592945098877
13 139 ndt        2.7977592945098877
13 140  t   tt    2.7977592945098877
13 141  et  t     2.7977592945098877
13 142 t   t      2.7977592945098877
13 143   e  t  t  2.7977592945098877
13 144 ndt        2.7977592945098877
13 145  t       e 2.7977592945098877
13 146 t       e  2.7977592945098877
13 147     t   t  2.7977592945098877
13 148  e     t   2.7977592945098877
13 149  et  t  tt 2.7977592945098877
13 150 t e  t  tt 2.7977592945098877
13 151     e      2.7977592945098877
13 152   e  tt    2.7977592945098877
13 153 odt  t   t 2.7977592945098877
13 154 tt  tt     2.7977592945098877
1

...

82 156 s immensit 0.5258713960647583
82 157 stgmensity 0.5258713960647583
82 158 tmmensity  0.5258713960647583
82 159 fmensity o 0.5258713960647583
82 160  eosity of 0.5258713960647583
82 161  hsity of  0.5258713960647583
82 162  gity of t 0.5258713960647583
82 163 dity of th 0.5258713960647583
82 164 sty of the 0.5258713960647583
82 165 fy of the  0.5258713960647583
82 166 o of the e 0.5258713960647583
82 167 oaf themeh 0.5258713960647583
82 168 to the eha 0.5258713960647583
82 169 n the ehao 0.5258713960647583
83 0 t you wnat 0.5229988694190979
83 1 otou wnat  0.5229988694190979
83 2 tou wnat t 0.5229988694190979
83 3 ou wnat to 0.5229988694190979
83 4 r wnat to  0.5229988694190979
83 5 pwnat to b 0.5229988694190979
83 6 toat to bu 0.5229988694190979
83 7 oat to bui 0.5229988694190979
83 8 dt to buil 0.5229988694190979
83 9 n to build 0.5229988694190979
83 10 oao build  0.5229988694190979
83 11 to cuild a 0.5229988694190979
83 12 o build ao 0.5229988694190979
83 13 rcuild aos 0.52299

86 40 i peaple t 0.489854633808136
86 41 ,peaple to 0.489854633808136
86 42 tnaple tog 0.489854633808136
86 43 ,ewle toge 0.489854633808136
86 44 nple toget 0.489854633808136
86 45 nle togeth 0.489854633808136
86 46 ,e thgethe 0.489854633808136
86 47 estogether 0.489854633808136
86 48 neogether  0.489854633808136
86 49 to ether t 0.489854633808136
86 50 o ether to 0.489854633808136
86 51 nkther to  0.489854633808136
86 52  ther to c 0.489854633808136
86 53 nher to co 0.489854633808136
86 54 oer to col 0.489854633808136
86 55 er to coll 0.489854633808136
86 56 n to colle 0.489854633808136
86 57  th collec 0.489854633808136
86 58 to collect 0.489854633808136
86 59 o bollect  0.489854633808136
86 60 ncollect a 0.489854633808136
86 61 tollect wo 0.489854633808136
86 62 tllect woo 0.489854633808136
86 63 nlect wood 0.489854633808136
86 64 eect wood  0.489854633808136
86 65 est wood a 0.489854633808136
86 66 nt wood an 0.489854633808136
86 67 t pood and 0.489854633808136
86 68 oaood and  0.4

90 0 t you wnat 0.569809079170227
90 1  tou wnath 0.569809079170227
90 2 tou wnatht 0.569809079170227
90 3 hu wnathto 0.569809079170227
90 4 n wnat to  0.569809079170227
90 5 pwnat to c 0.569809079170227
90 6 tnathao bu 0.569809079170227
90 7 oat to bui 0.569809079170227
90 8 dt to buil 0.569809079170227
90 9 nhto cuild 0.569809079170227
90 10 owo build  0.569809079170227
90 11 to cuild a 0.569809079170227
90 12 o build as 0.569809079170227
90 13 ncuild ass 0.569809079170227
90 14 tuild assh 0.569809079170227
90 15 uild asshi 0.569809079170227
90 16 pld asship 0.569809079170227
90 17 td asship, 0.569809079170227
90 18 e asship,  0.569809079170227
90 19  wnship, d 0.569809079170227
90 20 tnship, do 0.569809079170227
90 21 nship, don 0.569809079170227
90 22 thip, don' 0.569809079170227
90 23 sip, don't 0.569809079170227
90 24 ep, don't  0.569809079170227
90 25 t, don't a 0.569809079170227
90 26 , don't ar 0.569809079170227
90 27  bon't aru 0.569809079170227
90 28 ton't arum 0.56980907917

94 0 t you wnat 0.5289905667304993
94 1  tou wnat  0.5289905667304993
94 2 tou wnat t 0.5289905667304993
94 3 ou wnat to 0.5289905667304993
94 4 r wnat to  0.5289905667304993
94 5 mwnat to c 0.5289905667304993
94 6 toao to cu 0.5289905667304993
94 7 oat to cui 0.5289905667304993
94 8 dt ao buil 0.5289905667304993
94 9 t ao build 0.5289905667304993
94 10 owo build  0.5289905667304993
94 11 togcuild a 0.5289905667304993
94 12 o cuild an 0.5289905667304993
94 13 rcuild ans 0.5289905667304993
94 14 tutld ansh 0.5289905667304993
94 15 uild anshi 0.5289905667304993
94 16 mld anship 0.5289905667304993
94 17 te aoship, 0.5289905667304993
94 18 e anship,  0.5289905667304993
94 19  wnship, w 0.5289905667304993
94 20 tnship, wo 0.5289905667304993
94 21 tship, don 0.5289905667304993
94 22 thip, won' 0.5289905667304993
94 23 sip, don't 0.5289905667304993
94 24 ep, don't  0.5289905667304993
94 25 t, don't d 0.5289905667304993
94 26 , don't dr 0.5289905667304993
94 27  bon't dru 0.5289905667304993
94

97 128 efhem ta l 0.47173821926116943
97 129 toem ta lo 0.47173821926116943
97 130 oem ta lon 0.47173821926116943
97 131 er ta long 0.47173821926116943
97 132 r ta long  0.47173821926116943
97 133  uo long f 0.47173821926116943
97 134 to bong fo 0.47173821926116943
97 135 o bong for 0.47173821926116943
97 136 nlong for  0.47173821926116943
97 137 tong for t 0.47173821926116943
97 138 dnd for th 0.47173821926116943
97 139 n' for the 0.47173821926116943
97 140 d for the  0.47173821926116943
97 141  for the e 0.47173821926116943
97 142 thrdthe en 0.47173821926116943
97 143 or the end 0.47173821926116943
97 144 nkthe endl 0.47173821926116943
97 145  the endle 0.47173821926116943
97 146 toemendles 0.47173821926116943
97 147 oemendless 0.47173821926116943
97 148 erendless  0.47173821926116943
97 149 rendless i 0.47173821926116943
97 150 tndless im 0.47173821926116943
97 151 r less imm 0.47173821926116943
97 152 dless imme 0.47173821926116943
97 153  ess immen 0.47173821926116943
97 154 des i

In [15]:
Y.view(-1)

tensor([19,  2, 17,  ..., 24, 13, 14])